In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.formula.api import ols
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from math import sqrt

import scipy, matplotlib
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
import warnings


mainData = pd.read_csv('clean_filter_dataframe.csv')
sb.set() # set the default Seaborn style for graphics
mainData.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,...,Fighting,Misc,Platform.1,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
count,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,...,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000
mean,2007.369579,0.260723,0.139294,0.037282,0.048949,0.486453,70.496872,28.914995,7.216677,129.731607,...,0.058252,0.055016,0.061704,0.012945,0.075944,0.083495,0.115426,0.041640,0.163970,0.037972
std,4.069156,0.276439,0.183665,0.117220,0.068543,0.465979,12.903368,18.224541,1.381018,426.797953,...,0.234245,0.228036,0.240644,0.113049,0.264937,0.276659,0.319570,0.199786,0.370288,0.191149
min,1985.000000,0.000000,0.000000,0.000000,0.000000,0.010000,21.000000,3.000000,0.500000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2004.000000,0.070000,0.020000,0.000000,0.010000,0.140000,63.000000,15.000000,6.500000,11.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2007.000000,0.160000,0.060000,0.000000,0.020000,0.320000,72.000000,25.000000,7.500000,25.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2010.000000,0.360000,0.190000,0.010000,0.060000,0.690000,80.000000,40.000000,8.200000,77.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2016.000000,1.820000,1.580000,1.490000,1.180000,2.070000,98.000000,106.000000,9.600000,10665.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [2]:
data = mainData.drop(["Name", 'Developer', 'Publisher', 'Rating', 'Platform', 'Genre'],axis=1)
data.head()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,...,Fighting,Misc,Platform.1,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
0,2005.0,1.77,0.12,0.02,0.17,2.07,72.0,21.0,7.8,54.0,...,0,0,0,0,1,0,0,0,0,0
1,2006.0,0.92,0.93,0.00,0.22,2.07,49.0,5.0,5.2,24.0,...,0,0,0,0,0,0,0,1,0,0
2,2007.0,1.35,0.52,0.00,0.19,2.07,73.0,16.0,7.1,27.0,...,0,0,0,0,0,0,0,1,0,0
3,2014.0,0.36,1.38,0.02,0.31,2.07,71.0,85.0,6.1,1571.0,...,0,0,0,0,1,0,0,0,0,0
4,2009.0,1.48,0.39,0.00,0.19,2.06,83.0,68.0,7.9,51.0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
mainData_dummies = data
mainData_dummies.iloc[:,5:].head(5)
import numpy as np# Labels are the values we want to predict
labels = np.array(mainData_dummies['Global_Sales'])# Remove the labels from the features
# axis 1 refers to the columns
mainData_dummies=  mainData_dummies.drop('Global_Sales', axis = 1)# Saving feature names for later use
mainData_dummies = mainData_dummies.drop("NA_Sales", axis = 1)
mainData_dummies = mainData_dummies.drop("EU_Sales", axis = 1)
mainData_dummies = mainData_dummies.drop("JP_Sales", axis = 1)
mainData_dummies = mainData_dummies.drop("Other_Sales", axis = 1)

mainData_list = list(mainData_dummies.columns)# Convert to numpy array
mainData_dummies = np.array(mainData_dummies)
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(mainData_dummies, labels, test_size = 0.20, random_state = 42)

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)# Train the model on training data
rf.fit(X_train, y_train); 

predictions = rf.predict(X_test)# Calculate the absolute errors
errors_test = abs(predictions - y_test)# Print out the mean absolute error (mae)
print('Mean Absolute Error for Y_test:', round(np.mean(errors_test), 2), 'degrees.')

print('R squared training set:', rf.score(X_train, y_train))
print('R squared test set:', rf.score(X_test, y_test))

importances = list(rf.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(mainData_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Mean Absolute Error for Y_test: 0.28 degrees.
R squared training set: 0.9130670821792025
R squared test set: 0.3833395496981552
Variable: User_Count           Importance: 0.28812
Variable: Critic_Score         Importance: 0.13887
Variable: Critic_Count         Importance: 0.10838
Variable: User_Score           Importance: 0.1025
Variable: Year_of_Release      Importance: 0.09735
Variable: PC                   Importance: 0.07962
Variable: Sports               Importance: 0.01684
Variable: PS2                  Importance: 0.01531
Variable: Misc                 Importance: 0.01404
Variable: Action               Importance: 0.01186
Variable: Racing               Importance: 0.0113
Variable: DS                   Importance: 0.01044
Variable: Wii                  Importance: 0.01013
Variable: PS3                  Importance: 0.00907
Variable: Simulation           Importance: 0.00871
Variable: Shooter              Importance: 0.0087
Variable: Platform.1           Importance: 0.00823
Variable

In [4]:
value_Rating = 0
value_Genre = 0
value_Developer = 0
value_Year = 0
value_Name = 0
for name, number in feature_importances:
    if name in mainData.Rating.unique():
        value_Rating += number
    if name in mainData.Genre.unique():
        value_Genre += number
    if name in mainData.Developer.unique():
        value_Developer += number
    if name in mainData.Name.unique():
        value_Name += number
        
print(value_Rating)
print(value_Genre)
print(value_Developer)
print(value_Year)
print(value_Name)

0
0.09496
0
0
0
